### List comprehensions
**Where it is in your code**: 
```python
player_outline = [(x + player_pos[0], y + player_pos[1]) for x, y in player_outline]
```
**What it is**:

Say you have a list of numbers.
```python
numbers = [1, 2, 3, 4, 5, 6]
```
If you want to get all the even numbers from the list, one way is to create an empty list and then go through each entry of the original list using a for loop, check if it is even, and add it to the new list:
```python
evens = []
for num in numbers:
    if num % 2 == 0:
        evens.append(num)
```
A shorter way to write this is using a *list comprehension*. The code above could be shortened to: 
```python
evens = [num for num in numbers if num % 2 == 0]
```
Notice how the syntax is similar to how you would write the code without using comprehensions. The general form is:
```python
new_list = [expression for member in iterable if conditional]
```
**Advantages**: list comprehensions can sometimes make your code shorter and easier to write and read.

**Disadvantages**: list comprehensions might sometimes run more slowly than other methods of writing your code.

**Exercise**: write the following as a list comprehension.
```python
squares = []
for num in range(10):
    squares.append(num * num)
```
**Exercise**: what's the long way to write the `player_outline` code?

In [ ]:
# exercise: convert this to list comprehension
squares = []
for num in range(10):
    squares.append(num * num)

squares2 = [???]

### Unpacking lists
**Where it is in your code**: 
```python
player_rect = pygame.Rect(*player_pos, 50, 50)
```
**What it is**:

A way of passing the elements of lists (or tuples, sets, dictionaries) to functions individually, *not* as a whole list. For example:
```python
def greet(name, age):
    print(f"Hello, {name}. You are {age} years old.")

info = ["Alice", 30]
greet(*info)
```
The function `greet()` has two parameters, `name` and `age`. If we write `greet(info)` without the `*`, Python would raise an error, because you are passing only a single argument, `info`, which is a list. However, by writing `greet(*info)`, you tell Python to *unpack* the list `info`: give the individual entries of `info` to the function.

**Exercise**
Use the `*` operator to sum the entries of `numbers` using the function below.
```python
numbers = [4, 7, 1]

def sum_three_numbers(a, b, c):
    return sum(a, b, c)

result = ...
```

In [3]:
import pygame
import random
import sys

pygame.init()

screen = pygame.display.set_mode((800, 600))
pygame.display.set_caption("Sprites")

# load images
player_img = pygame.image.load("zelda.png").convert_alpha()
player_img = pygame.transform.scale(player_img, (50, 50))
background_img = pygame.image.load("white.png")
background_img = pygame.transform.scale(background_img, (800,600))
enemy_img = pygame.image.load("ganon.png").convert_alpha()
enemy_img = pygame.transform.scale(enemy_img, (100,100))
expl_img = pygame.image.load("explosion.png").convert_alpha()
expl_img = pygame.transform.scale(expl_img, (100,100))

player_mask = pygame.mask.from_surface(player_img)
enemy_mask = pygame.mask.from_surface(enemy_img)

player_pos = [100, 100] # [x,y]
enemy_pos = [300, 300]

speed = 2
enemy_speed = 1
enemy_direction = [random.choice([-1,1]), random.choice([-1,1])]
lives = 3
game_duration = 30 * 1000 # 30 seconds in ms
start_time = pygame.time.get_ticks()
font = pygame.font.SysFont("Arial", 30)

running = True
game_over = False
while running:
    while game_over:
        if lives > 0:
            screen.fill((0, 255, 0)) # green
            result_text = font.render("You win! Press 'R' to play again or 'Q' to quit.", True, (0,0,0))
        else:
            screen.fill((255, 0, 0)) # red
            result_text = font.render("You lose! Press 'R' to play again or 'Q' to quit.", True, (0,0,0))
        final_score_text = font.render(f"Lives remaining: {lives}", True, (0,0,0))
        # // is floor division
        screen.blit(result_text, (screen.get_width() // 2 - result_text.get_width() // 2, 200))
        screen.blit(final_score_text, (screen.get_width() // 2 - final_score_text.get_width() // 2, 250))
        pygame.display.flip()

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                game_over = False
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_q:
                    running = False
                    game_over = False
                if event.key == pygame.K_r: # reset constants
                    game_over = False
                    player_pos = [100, 100]
                    enemy_pos = [300, 300]
                    lives = 3
                    start_time = pygame.time.get_ticks()

    elapsed_time = pygame.time.get_ticks() - start_time
    remaining_time = max(0, (game_duration - elapsed_time) // 1000)

    if remaining_time == 0 or lives == 0:
        game_over = True
    elif remaining_time < 10:
        enemy_img = expl_img
        
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    keys = pygame.key.get_pressed()
    # arrow keys for circle
    if keys[pygame.K_LEFT]:
        player_pos[0] -= speed
    if keys[pygame.K_RIGHT]:
        player_pos[0] += speed
    if keys[pygame.K_UP]:
        player_pos[1] -= speed
    if keys[pygame.K_DOWN]:
        player_pos[1] += speed

    # move enemy
    enemy_pos[0] += enemy_direction[0] * enemy_speed
    enemy_pos[1] += enemy_direction[1] * enemy_speed

    # bounce enemy off the walls
    if enemy_pos[0] <= 0 or enemy_pos[0] + 100 >= 800:
        enemy_direction[0] *= -1
    if enemy_pos[1] <= 0 or enemy_pos[1] + 100 >= 600:
        enemy_direction[1] *= -1

    # limit player to the window
    if player_pos[0] < 0:
        player_pos[0] = 0
    elif player_pos[0] > 800 - 50:
        player_pos[0] = 800 - 50
    if player_pos[1] < 0:
        player_pos[1] = 0
    elif player_pos[1] > 600 - 50:
        player_pos[1] = 600 - 50

    screen.blit(background_img, (0,0))
    screen.blit(player_img, player_pos)
    screen.blit(enemy_img, enemy_pos)

    # draw rectangles
    player_rect = pygame.Rect(*player_pos, 50, 50) # unpacking list
    enemy_rect = pygame.Rect(*enemy_pos, 100, 100)
    offset = (enemy_rect.left - player_rect.left, enemy_rect.top - player_rect.top)
    collision = player_mask.overlap(enemy_mask, offset)

    # pygame.draw.rect(screen, (0, 255, 0), player_rect, 3)
    # pygame.draw.rect(screen, (0, 255, 0), enemy_rect, 3)
    player_outline = player_mask.outline()
    player_outline = [(x + player_pos[0], y + player_pos[1]) for x, y in player_outline] # list comprehension
    pygame.draw.lines(screen, (0,255,0), True, player_outline, 3)

    enemy_outline = enemy_mask.outline()
    enemy_outline = [(x + enemy_pos[0], y + enemy_pos[1]) for x, y in enemy_outline]
    pygame.draw.lines(screen, (0,255,0), True, enemy_outline, 3)

    if collision:
        # pygame.draw.rect(screen, (255,0,0), player_rect, 3)
        pygame.draw.lines(screen, (255, 0, 0), True, player_outline, 3)
        lives -= 1
        screen.blit(font.render("Ganon hit you!", True, (255, 0, 0)), player_pos)
        pygame.display.flip()
        pygame.time.wait(1000)
        start_time += 1000
        player_pos = [random.randint(0, 800), random.randint(0, 600)]

    lives_text = font.render(f"Lives: {lives}", True, (0,0,0))
    time_text = font.render(f"Time: {remaining_time}s", True, (0,0,0))
    screen.blit(lives_text, (10, 10))
    screen.blit(time_text, (10, 50))

    pygame.display.flip()
    pygame.time.delay(10)

quit()